**4. CARGA**

O código abaixo implementa a etapa de ingestão e padronização da camada Bronze em um pipeline de dados. Ele carrega a tabela bruta existente no ambiente, aplica uma normalização automática aos nomes das colunas para garantir consistência e conformidade com boas práticas, acrescenta um timestamp técnico que registra o momento da ingestão e, ao final, grava o resultado em uma tabela Delta no schema default.

In [0]:
import unicodedata
from pyspark.sql import functions as F

# Normalização das colunas

def normalize_column(col_name):
    col = ''.join(
        c for c in unicodedata.normalize('NFD', col_name)
        if unicodedata.category(c) != 'Mn'
    )
    col = col.replace(" ", "_")
    col = ''.join(c for c in col if c.isalnum() or c == "_")
    return col.lower()



# Carregar tabela RAW

df_raw = spark.table("default.contratos_publicos_2020_2024")



# Normalizar nomes das colunas

new_cols = [normalize_column(c) for c in df_raw.columns]
df_bronze = df_raw.toDF(*new_cols)


# Coluna técnica: timestamp de ingestão

df_bronze = df_bronze.withColumn("raw_ingestion_timestamp", F.current_timestamp())



# Criar Tabela Bronze Delta

df_bronze.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.contratos_publicos_bronze")

# Visualização Tabela Bronze

display(df_bronze.limit(10))


numero_do_contrato,objeto,fundamento_legal,modalidade_compra,situacao_contrato,codigo_orgao_superior,nome_orgao_superior,codigo_orgao,nome_orgao,codigo_ug,nome_ug,data_assinatura_contrato,data_publicacao_dou,data_inicio_vigencia,data_fim_vigencia,codigo_contratado,nome_contratado,valor_inicial_compra,valor_final_compra,numero_licitacao,codigo_ug_licitacao,nome_ug_licitacao,codigo_modalidade_compra_licitacao,modalidade_compra_licitacao,arquivo_origem,raw_ingestion_timestamp
124882022,"Objeto: CONSTITUI OBJETO DESTE INSTRUMENTO A CONTRATAÇÃO DA FINATEC PARA APOIAR A EXECUÇÃO E O DESENVOLVIMENTO DO PROJETO DE PESQUISA INTITULADO 'ELABORAÇÃO DO PLANO DE AÇÃO CIÊNCIA PARA BIODIVERSIDADE E SERVIÇOS ECOSSISTÊMICOS', ORIUNDO DO TERMO DE EXECUÇÃO DESCENTRALIZADA Nº 26717520220006-000545, CELEBRADO ENTRE O MINISTÉRIO DA CIÊNCIA, TECNOLOGIA E INOVAÇÕES E A UNIVERSIDADE DE BRASÍLIA, EM 07/10/2022. A CONTRATANTE INDICA, DESDE JÁ, COMO RESPONSÁVEL PELA COORDENAÇÃO E ACOMPANHAMENTO DAS ATIVIDADES DO PROJETO,",null,Dispensa de Licitação,Não se aplica,26000,Ministério da Educação,26271,Fundação Universidade de Brasília,154040,FUNDACAO UNIVERSIDADE DE BRASILIA - UNB,2023-01-09,2023-03-22,2023-01-09,2024-03-31,37116704000134,FUNDACAO DE EMPREENDIMENTOS CIENTIFICOS E TECNOLOGICOS,"182000,0000","182000,0000",124882022,154040.0,FUNDACAO UNIVERSIDADE DE BRASILIA - UNB,6.0,Dispensa de Licitação,202303_Compras.csv,2025-12-20T19:51:28.086Z
52022,Objeto: CONTRATAÇÃO CADASTRADO NA UNIDADE EQUIVOCADA,null,Dispensa de Licitação,Não se aplica,32000,Ministério de Minas e Energia,29208,Companhia de Pesquisa de Recursos Minerais,495370,COMPANHIA DE PESQUISA DE RECURSOS MINERAIS,2023-03-24,2023-03-27,2023-03-24,2024-03-24,09400465000104,WEBJUR PROCESSAMENTO DE DADOS LTDA,"480,0000","480,0000",1232022,495130.0,COMPANHIA DE PESQUISA DE RECURSOS MINERAIS,6.0,Dispensa de Licitação,202303_Compras.csv,2025-12-20T19:51:28.086Z
492023,"Objeto: CONTRATAÇÃO DE EMPRESA ESPECIALIZADA NA PRESTAÇÃO DE SERVIÇOS DE ACOMPANHAMENTO, POR MEIO DE LEITURA, EM NÍVEL NACIONAL, DOS DIÁRIOS DE JUSTIÇA DE TODAS AS UNIDADES DA FEDERAÇÃO E DO DIÁRIO OFICIAL DA UNIÃO (DOU), NA FORMA ELETRÔNICA, BEM COMO DE ENTREGA DIÁRIA, VIA E-MAIL, DAS PUBLICAÇÕES RELATIVAS AOS PROCESSOS JUDICIAIS E DAS PUBLICAÇÕES REALIZADAS NO DIÁRIO OFICIAL DA UNIÃO (DOU), AMBOS EM NOME DA COMPANHIA DE PESQUISA DE RECURSOS MINERAIS - CPRM.",null,Dispensa de Licitação,Não se aplica,32000,Ministério de Minas e Energia,29208,Companhia de Pesquisa de Recursos Minerais,495130,COMPANHIA DE PESQUISA DE RECURSOS MINERAIS,2023-03-24,2023-03-27,2023-03-24,2026-03-24,09400465000104,WEBJUR PROCESSAMENTO DE DADOS LTDA,"480,0000","524,5000",1232022,495130.0,COMPANHIA DE PESQUISA DE RECURSOS MINERAIS,6.0,Dispensa de Licitação,202303_Compras.csv,2025-12-20T19:51:28.086Z
32023,"Objeto: CONTRATAÇÃO DE SERVIÇOS CONTINUADOS DE VIGILÂNCIA ARMADA, COM DISPONIBILIZAÇÃO DE MÃO DE OBRA EM REGIME DE DEDICAÇÃO EXCLUSIVA, FORNECIMENTO DE UNIFORMES, MATERIAIS E EQUIPAMENTOS NECESSÁRIOS, PARA A PRESTAÇÃO DOS SERVIÇOS A SEREM EXECUTADOS NAS DEPENDÊNCIAS DO CENTRO NACIONAL DE PESQUISA E CONSERVAÇÃO DA BIODIVERSIDADE MARINHA DO SUDESTE E SUL - CEPSUL/SC.",null,Pregão,Não se aplica,44000,Ministério do Meio Ambiente e Mudança do Cl,44207,Instituto Chico Mendes de Conservação da Bi,443033,INSTITUTO CHICO MENDES - SEDE,2023-03-08,2023-03-13,2023-03-08,2026-03-08,11933418000178,SEGPLUS - SISTEMAS DE SEGURANCA LTDA,"147528,2800","173703,6000",12022,443040.0,ICMBIO DIAC,5.0,Pregão,202303_Compras.csv,2025-12-20T19:51:28.086Z
102023,Objeto: PRESTAÇÃO DE SERVIÇO CONTINUADO DE LAVANDERIA.,null,Pregão - Registro de Preço,Não se aplica,52000,Ministério da Defesa,52121,Comando do Exército,160413,BASE ADM DA GUARNICAO DE SANTA MARIA,2023-03-20,2023-03-22,2023-03-20,2026-03-19,01593873000118,CLEVISON CARNELOZO DA COSTA,"746467,7500","802190,4000",492022,160413.0,BASE ADM DA GUARNICAO DE SANTA MARIA,9999.0,Pregão - Registro de Preço,202303_Compras.csv,2025-12-20T19:5

Para construir a camada silver, o objetivo era filtrar apenas as colunas úteis, padronizá-las e definir os tipos de dado correto de cada uma. Assim, foi definido uma função para normalizar textos, removendo acentos, trocando "ç" por "c", eliminando espaços desnecessários e convertendo tudo para maiúsculas. Durante essa transformação, ocorre a normalização dos textos das colunas "objeto" e "nome_contratado", conversão da coluna de data considerando dois formatos possíveis e ajuste da coluna de valor para o formato numérico correto, removendo pontos e substituindo vírgulas por ponto antes de convertê-la para double. Por fim, foram removidos registros duplicados.

In [0]:
from pyspark.sql import functions as F

# Mapa de substituição para remover acentos

acentos = "áàãâäéèêëíìîïóòõôöúùûüçÁÀÃÂÄÉÈÊËÍÌÎÏÓÒÕÔÖÚÙÛÜÇ"
sem_acentos = "AAAAAEEEEIIIIOOOOOUUUUCAAAAAEEEEIIIIOOOOOUUUUC"

def normalize_upper(col):
    return (
        F.upper(
            F.trim(
                F.translate(col, acentos, sem_acentos)  # remove acentos
            )
        )
    )

# Transformando as colunas

df_bronze = spark.table("default.contratos_publicos_bronze")

df_silver_pre = (
    df_bronze
    .select(
        F.col("numero_do_contrato").cast("string"),

        # OBJETO normalizado
        normalize_upper(F.col("objeto")).alias("objeto"),

        F.col("modalidade_compra").cast("string"),
        F.col("nome_orgao").cast("string"),

        # Data
        F.coalesce(
            F.to_date("data_publicacao_dou", "yyyy-MM-dd"),
            F.to_date("data_publicacao_dou", "dd/MM/yyyy")
        ).alias("data_publicacao_dou"),

        # NOME CONTRATADO normalizado
        normalize_upper(F.col("nome_contratado")).alias("nome_contratado"),

        # Valor
        F.regexp_replace(
            F.regexp_replace("valor_final_compra", r"\.", ""),
            ",", "."
        )
        .cast("double")
        .alias("valor_final_compra")
    )
)

# Retirar duplicatas

df_silver = df_silver_pre.dropDuplicates()

# Criar Tabela Silver Delta

df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.contratos_publicos_silver")

# Visualização Tabela Silver

display(df_silver.limit(10))


numero_do_contrato,objeto,modalidade_compra,nome_orgao,data_publicacao_dou,nome_contratado,valor_final_compra
22020,INFORMACAO PROTEGIDA POR SIGILO NOS TERMOS DA LEGISLACAO VIGENTE,Dispensa de Licitação,Polícia Federal,2020-01-29,SIGILOSO,1533441.96
362019,INFORMACAO PROTEGIDA POR SIGILO NOS TERMOS DA LEGISLACAO VIGENTE,Pregão - Registro de Preço,Polícia Federal,2020-01-07,SIGILOSO,66807.86
282019,INFORMACAO PROTEGIDA POR SIGILO NOS TERMOS DA LEGISLACAO VIGENTE,Pregão - Registro de Preço,Polícia Federal,2020-01-28,SIGILOSO,66300.0
352019,INFORMACAO PROTEGIDA POR SIGILO NOS TERMOS DA LEGISLACAO VIGENTE,Dispensa de Licitação,Polícia Federal,2020-01-24,SIGILOSO,17548.0
12020,INFORMACAO PROTEGIDA POR SIGILO NOS TERMOS DA LEGISLACAO VIGENTE,Pregão - Registro de Preço,Polícia Federal,2020-01-24,SIGILOSO,233377.82
482019,INFORMACAO PROTEGIDA POR SIGILO NOS TERMOS DA LEGISLACAO VIGENTE,Pregão,Polícia Federal,2020-01-27,SIGILOSO,288959.0
492019,INFORMACAO PROTEGIDA POR SIGILO NOS TERMOS DA LEGISLACAO VIGENTE,Pregão,Polícia Federal,2020-01-23,SIGILOSO,303882.0
502019,INFORMACAO PROTEGIDA POR SIGILO NOS TERMOS DA LEGISLACAO VIGENTE,Pregão,Polícia Federal,2020-01-23,SIGILOSO,1150790.0
462019,INFORMACAO PROTEGIDA POR SIGILO NOS TERMOS DA LEGISLACAO VIGENTE,Pregão,Polícia Federal,2020-01-23,SIGILOSO,845900.0
12020,INFORMACAO PROTEGIDA POR SIGILO NOS TERMOS DA LEGISLACAO VIGENTE,Pregão - Registro de Preço,Polícia Federal,2020-01-30,SIGILOSO,3360.0


Na criação da camada gold, o escopo consiste em substituir a coluna "objeto" por "grupo_objeto" e aplicar regras de qualidade de dados para obter a base de dados mais limpa e consistente possível. Para isso, foram definidas quatro listas de palavras-chave que representam diferentes categorias de objetos: bens patrimoniais, materiais de consumo, obras e serviços. Em seguida, cria uma função auxiliar que verifica se uma coluna contém qualquer uma das palavras dessas listas. Depois disso, cria-se uma nova coluna chamada grupo_objeto, classificando cada registro conforme a presença dessas palavras no campo “objeto”. A parte das regras de qualidade define um conjunto de critérios que cada registro precisa atender para ser considerado válido. Esses critérios verificam cinco pontos principais: se o número do contrato contém apenas dígitos e não é composto apenas de zeros; se a modalidade de compra pertence a uma lista de modalidades permitidas; se a data de publicação está dentro de um intervalo pré-definido (entre 2020 e 2024); se o valor final da compra é maior que zero; e se alguma coluna possui campos nulos. Com base nessas regras, gera duas novas colunas: "status_qualidade" (OK ou ERRO) e "motivo_erro" que descreve quais critérios foram violados, sendo elas utilizadas apenas na tabela de registros inválidos. A partir dessa classificação, o código separa os dados em dois conjuntos: um conjunto Gold contendo apenas registros válidos (destinado a análises de negócio) e um conjunto inválido contendo somente os registros com erro.

In [0]:
# Listas palavras-chave

bens = [
    "VEICULO","CARRO","CAMINHAO","ONIBUS","MOTO","MOTOCICLETA","VEICULOS",
    "COMPUTADOR","NOTEBOOK","LAPTOP","DESKTOP","CPU","IMPRESSORA","SCANNER",
    "SERVIDOR","MONITOR","TECLADO","MOUSE","ESTABILIZADOR","NOBREAK",
    "MESA","CADEIRA","ARMARIO","ESTANTE","MOVEIS","MOBILIARIO",
    "GELADEIRA","FREEZER","FRIGOBAR","AR CONDICIONADO","CONDICIONADOR DE AR",
    "EQUIPAMENTO","EQUIPAMENTOS","APARELHO","APARELHOS","MAQUINA","MAQUINAS",
    "FERRAMENTA","FERRAMENTAS","PERMANENTE","PATRIMONIAL","BENS"
]

materiais = [
    "PAPEL","CANETA","LAPIS","BORRACHA","CLIPS","MATERIAL","MATERIAIS",
    "EXPEDIENTE","INSUMO","INSUMOS",
    "ALIMENTO","ALIMENTOS","GENERO","CESTA BASICA","CAFE","AGUA",
    "LIMPEZA","DETERGENTE","ALCOOL","DESINFETANTE","SABAO","SANEANTES",
    "COMBUSTIVEL","GASOLINA","DIESEL","OLEO","ETANOL",
    "MEDICAMENTO","MEDICAMENTOS","FARMACEUTICO","FARMACO",
    "SERINGA","LUVA","MATERIAL HOSPITALAR","MATERIAL DE CONSUMO",
    "PANO","EPIS","DESCARTAVEL","COPOS"
]

obras = [
    "OBRA","OBRAS","CONSTRUCAO","REFORMA","AMPLIACAO","ENGENHARIA",
    "PAVIMENTACAO","ASFALTO","ASFALTICA",
    "MANUTENCAO PREDIAL","MANUTENCAO DE EDIFICACAO",
    "ESTRUTURA","EDIFICACAO","PREDIO","OBRAS CIVIS","CIVIL",
    "TERRAPLENAGEM","DRENAGEM","CALCADA","MURO","ALVENARIA",
    "INSTALACAO ELETRICA","INSTALACAO HIDRAULICA","PROJETO EXECUTIVO",
    "RECUPERACAO","RESTAURO","CONSTRUCOES"
]

servicos = [
    "SERVICO","SERVICOS","CONSULTORIA","ASSESSORIA","MANUTENCAO","SUPORTE",
    "LIMPEZA","VIGILANCIA","SEGURANCA","PORTARIA",
    "LOCACAO","ALUGUEL","CONTRATACAO","RETROFIT",
    "TREINAMENTO","CAPACITACAO",
    "DESENVOLVIMENTO","SOFTWARE","SISTEMA","TI",
    "TRANSPORTE","FRETE","ENTREGA","COLETA",
    "PUBLICIDADE","PROPAGANDA","DIVULGACAO",
    "TERCEIRIZACAO","ADMINISTRACAO",
    "ANALISE","PERICIA","AUDITORIA"
]

# Função auxiliar

from pyspark.sql import functions as F

def contains_any(col, keywords):
    return F.expr(" OR ".join([f"INSTR({col}, '{k}') > 0" for k in keywords]))


# Carregar Silver

df_silver = spark.table("default.contratos_publicos_silver")


# Classificação do grupo do objeto

df_class = df_silver.withColumn(
    "grupo_objeto",
    F.when(contains_any("objeto", bens), "BENS PATRIMONIAIS")
     .when(contains_any("objeto", materiais), "MATERIAIS")
     .when(contains_any("objeto", obras), "OBRAS")
     .when(contains_any("objeto", servicos), "SERVIÇOS")
     .otherwise("OUTROS")
)


# Regras de Qualidade

modalidades_validas = [
"Convite", "Tomada de Preços", "Concorrência", "Concorrência Internacional", "Pregão", "Dispensa de Licitação", "Inexigibilidade de Licitação", "Concurso",
"Tomada de Preços por Técnica e Preço", "Concorrência por Técnica e Preço", "Concorrência Internacional por Técnica e Preço", "Pregão - Registro de Preço", "Sem Informação"
]

dq_numero = F.col("numero_do_contrato").rlike(r"^(?!0+$)\d+$")
dq_modalidade = F.col("modalidade_compra").isin(modalidades_validas)
dq_data = (
    (F.col("data_publicacao_dou") >= F.lit("2020-01-01")) &
    (F.col("data_publicacao_dou") <= F.lit("2024-12-31"))
)
dq_valor = (F.col("valor_final_compra") > 0)
dq_null = F.expr(" OR ".join([f"{c} IS NULL" for c in df_silver.columns]))


# Status e Motivo de Erro

df_quality = df_class.withColumn(
    "status_qualidade",
    F.when(dq_numero & dq_modalidade & dq_data & dq_valor & ~dq_null, "OK")
     .otherwise("ERRO")
).withColumn(
    "motivo_erro",
    F.concat_ws("; ", 
                F.when(~dq_numero, "Número do Contrato inválido"), 
                F.when(~dq_modalidade, "Modalidade de compra inválida"), 
                F.when(~dq_data, "Data de publicação fora do intervalo"), 
                F.when(~dq_valor, "Valor final da compra inválido"),
                F.when(dq_null, "Campos obrigatórios nulos") ) )

# ---------------------------
# Criar outputs:
# GOLD = somente OK
# INVÁLIDO = somente ERRO
# ---------------------------

df_gold = (
    df_quality
        .filter("status_qualidade = 'OK'")
        .select(
            "numero_do_contrato",
            "grupo_objeto",
            "modalidade_compra",
            "nome_orgao",
            "data_publicacao_dou",
            "nome_contratado",
            "valor_final_compra"
        )
)

df_invalido = df_quality.filter("status_qualidade = 'ERRO'")

# Salvar Tabela Gold
df_gold.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.contratos_publicos_gold")

# Salvar Tabela Inválido
df_invalido.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.contratos_publicos_invalidos")

# Visualização Tabela Gold e Tabela Inválido

display(df_gold.limit(10))

display(df_invalido.limit(10))


numero_do_contrato,grupo_objeto,modalidade_compra,nome_orgao,data_publicacao_dou,nome_contratado,valor_final_compra
183002019,BENS PATRIMONIAIS,Sem Informação,Companhia de Desenvolvimento dos Vales do Sã,2020-01-29,BRE - EMPRESA BRASILEIRA DE EQUIPAMENTOS LTDA,708776.0
82019,BENS PATRIMONIAIS,Pregão - Registro de Preço,Instituto Nacional de Colonização e Reforma,2020-01-08,METDATA TECNOLOGIA DA INFORMACAO LTDA,39999.6
12020,OUTROS,Pregão - Registro de Preço,Polícia Rodoviária Federal,2020-01-06,CONDOR S/A INDUSTRIA QUIMICA,7000.0
162019,BENS PATRIMONIAIS,Dispensa de Licitação,Ministério da Saúde - Unidades com vínculo,2020-01-20,MZ COMERCIO E SERVICOS LTDA,179461.4
12020,SERVIÇOS,Pregão - Registro de Preço,"Instituto Federal de Educação, Ciência e T",2020-02-10,STARR INTERNATIONAL BRASIL SEGURADORA S.A.,8900.0
62020,MATERIAIS,Pregão,"Instituto Federal de Educação, Ciência e T",2020-02-03,PROENG TECNOLOGIA E SERVICOS LTDA,272211.24
182020,BENS PATRIMONIAIS,Pregão - Registro de Preço,Fundação Universidade Federal do Pampa,2020-02-05,CINCO ESTRELAS TRANSPORTES E LOGISTICA LTDA,67590.6
500532020,BENS PATRIMONIAIS,Sem Informação,Banco Central do Brasil - Orçamento Fiscal e,2020-03-10,ARTESANA DIVISORIAS E FORROS LTDA,323.88
102019,SERVIÇOS,Dispensa de Licitação,Comando do Exército,2020-03-26,TELEFONICA BRASIL S.A.,3861.82
12020,OUTROS,Inexigibilidade de Licitação,Fundação Oswaldo Cruz,2020-03-16,CHARLES RIVER DETECCAO MICROBIANA E DE ENDOTOXINA PARTICIPACOES LTDA.,343896.0


numero_do_contrato,objeto,modalidade_compra,nome_orgao,data_publicacao_dou,nome_contratado,valor_final_compra,grupo_objeto,status_qualidade,motivo_erro
122022,OBJETO: CONTRATACAO DE LEILOEIRO PUBLICO OFICIAL PARA ALIENACAO DE ATIVOS - AREA DE ABRANGENCIA: ESTADO DE ALAGOAS,Inexigibilidade de Licitação,Ministério da Justiça e Segurança Pública,2022-01-24,PAULO CEZAR ROCHA TEIXEIRA,0.0,SERVIÇOS,ERRO,Valor final da compra inválido
322019,"OBJETO: SERVICO COMUMS DE ENGENHARIA INERENTE A MANUTENCAO PREDIAL PREVENTIVA E CORRETIVA DOSBENS IMOVEIS, COM FORNECIMENTO DE PECAS, EQUIPAMENTOS E MATERIAIS E MAO DE OBRA.",Pregão - Registro de Preço,Fundação Universidade Federal da Grande Dou,2020-02-12,MURANO CONSTRUCOES LTDA,0.0,BENS PATRIMONIAIS,ERRO,Valor final da compra inválido
802020,"OBJETO: LICENCA, EM CARATER EXCLUSIVO PARA TV ABERTA, QUE A CBF CONCEDE A TV BRASIL, PARA O TERRITORIO BRASILEIRO, DOS DIREITOS DE TRANSMISSAO DE 2 (DOIS) JOGOS DE CADA RODADA DA SERIE D DO CAMPEONATO, INCLUINDO AS 2 (DUAS) PARTIDAS FINAIS, TOTALIZANDO 42 (QUARENTA E DUAS) PARTIDAS DO CAMPEONATO, DURANTE A TEMPORADA 2020.",Sem Informação,Empresa Brasil de Comunicação,2020-12-16,CONFEDERACAO BRASILEIRA DE FUTEBOL - CBF,0.0,SERVIÇOS,ERRO,Valor final da compra inválido
62021,"OBJETO: CESSAO, PELA COMODANTE AO COMODATARIO, SEM QUALQUER ONUS, NOS TERMOS DO ARTIGO 579 E SEGUINTES DO CODIGO CIVIL VIGENTE, DE 1 CLIPADOR VIA ENDOSCOPICA E 1 CLIPADOR PARA CIRURGIA ABERTA, CONFORME ESPECIFICACAO DESCRITA NA PROPOSTA COMERCIAL E NO EDITAL DE LICITACAO.",Sem Informação,Empresa Brasileira de Serviços Hospitalares,2021-04-19,TRADEHOSP COMERCIO DE PRODUTOS PARA SAUDE LTDA,0.0,OBRAS,ERRO,Valor final da compra inválido
22022,OBJETO: SERVICO MEDICO NA AREA DE CIRUGIA GERAL,Sem Informação,Comando da Marinha,2022-02-22,CLINICA DR. MARCO RAPHAEL LTDA,0.0,SERVIÇOS,ERRO,Valor final da compra inválido
322018,"OBJETO: CESSAO DE USO DE BEM IMOVEL PROPRIO NACIONAL SITUADO NO ENDERECO TERREO DO BLOCO H DO QUARTEL GENERAL DO EXERCITO SOB ADMINISTRACAO DA BASE ADMINISTRATIVA DO QUARTEL-GENERAL DO EXERCITO A FUNDACAO HABITACIONAL DO EXERCITO (FHE), MEDIANTE O REGIME DE EXERCICIO DE ATIVIDADES DE APOIO, COM A FINALIDADE DE FUNCIONAMENTO DE POSTO DE ATENDIMENTO",Inexigibilidade de Licitação,Comando do Exército,2022-09-13,FUNDACAO HABITACIONAL DO EXERCITO - FHE,0.0,SERVIÇOS,ERRO,Valor final da compra inválido
102019,"OBJETO: CONCESSAO NAO ONEROSA DE USO DE UM QUIOSQUE CONTAINER MOVEL, COM AREA DE 14,64M², DESTINADO A CONTRATACAO DE EMPRESA ESPECIALIZADA NA EXPLORACAO COMERCIAL DE CANTINA, SOB O CRITERIO DE MAIOR DESCONTO, SOBRE O CARDAPIO MINIMO, COM O FORNECIMENTO DE TODO O MATERIAL, EQUIPAMENTOS E SERVICOS NECESSARIOS PARA A EXECUCAO TOTAL DO SERVICO, PARA ATENDER A DEMANDA DO INSTITUTO FEDERAL CATARINENSE - CAMPUS FRAIBURGO.",Pregão,"Instituto Federal de Educação, Ciência e T",2020-08-25,33.861.274 DANIELA GUZZI DA ROSA,0.0,BENS PATRIMONIAIS,ERRO,Valor final da compra inválido
4022022,"OBJETO: O OBJETO DO PRESENTE INSTRUMENTO E A CONTRATACAO DE SERVICOS CONTINUADOS DE PESSOA JURIDICA PRESTADORA DE SERVICOS ESPECIALIZADOS EM APOIO A GESTAO DE FROTA, INCLUINDO A MANUTENCAO PREVENTIVA E CORRETIVA, E O FORNECIMENTO DE PECAS, EM REDE DE ESTABELECIMENTOS CREDENCIADOS, UTILIZANDO-SE DE TECNOLOGIA DA INFORMACAO, NA ADMINISTRACAO E CONTROLE DAS FROTAS DE VEICULOS OFICIAIS DAS ORGANIZACOES MILITARES SUBORDINADAS AO COMANDO DO 32 DISTRITO NAVAL (COM32DN), PARA ATENDIMENTO DAS SUAS NECESSIDADES, QUE SE",Pregão - Registro de Preço,Comando da Marinha,2022-08-26,LINK CARD ADMINISTRADORA DE BENEFICIOS LTDA,0.0,BENS PATRIMONIAIS,ERRO,Valor final da compra inválido
572023,"OBJETO: DISPONIBILIZACAO, PARA O ITEM N.º 7 DO TERMO DE REFERENCIA, DE MAQUINA PARA AUTOTRANSFUSAO EM REGIME DE COMODATO",Pregão - Registro de Preço,Empresa Brasileira de Serviços Hospitalares,2023-08-03,MEDICAL LIFE COMERCIO DE PRODUTOS HOSPITALARES LTDA,0.0,BENS PATRIMONIAIS,ERRO,Valor final da compra inválido
1812023,"OBJETO: EMPRES

Por fim, foi gerado uma tabela dimensão calendário completa, criando todas as datas entre 2000 e 2050 e adicionando diversos atributos derivados, como ano, mês, dia, trimestre, semana do ano e combinações como Ano-Mês e Ano-Semana, nomes dos meses e dias da semana em português, tanto completos quanto abreviados.

In [0]:
from pyspark.sql import functions as F
from datetime import datetime

# GERAR FAIXA DE DATAS

inicio = "2000-01-01"
fim     = "2050-12-31"

dias = (datetime.strptime(fim, "%Y-%m-%d") - datetime.strptime(inicio, "%Y-%m-%d")).days

df = (
    spark.range(0, dias + 1)
    .withColumn("id_int", F.col("id").cast("int"))
    .withColumn("Data", F.expr(f"date_add('{inicio}', id_int)"))
    .drop("id", "id_int")
)


# CAMPOS BÁSICOS

df_cal = (
    df
    .withColumn("Ano", F.year("Data"))
    .withColumn("Mes", F.month("Data"))
    .withColumn("Dia", F.dayofmonth("Data"))
    .withColumn("DiaSemana", F.dayofweek("Data"))
    .withColumn("Trimestre", F.quarter("Data"))
    .withColumn("SemanaAno", F.weekofyear("Data"))
    .withColumn("AnoMes", F.date_format("Data", "yyyy-MM"))
    .withColumn("AnoSemana", F.concat_ws("-", F.year("Data"), F.weekofyear("Data")))
)


# NOMES DOS MESES (PT-BR)

df_cal = df_cal.withColumn(
    "NomeMes",
    F.when(F.col("Mes") == 1, "Janeiro")
     .when(F.col("Mes") == 2, "Fevereiro")
     .when(F.col("Mes") == 3, "Março")
     .when(F.col("Mes") == 4, "Abril")
     .when(F.col("Mes") == 5, "Maio")
     .when(F.col("Mes") == 6, "Junho")
     .when(F.col("Mes") == 7, "Julho")
     .when(F.col("Mes") == 8, "Agosto")
     .when(F.col("Mes") == 9, "Setembro")
     .when(F.col("Mes") == 10, "Outubro")
     .when(F.col("Mes") == 11, "Novembro")
     .otherwise("Dezembro")
)

df_cal = df_cal.withColumn(
    "NomeMesAbrev",
    F.when(F.col("Mes") == 1, "Jan")
     .when(F.col("Mes") == 2, "Fev")
     .when(F.col("Mes") == 3, "Mar")
     .when(F.col("Mes") == 4, "Abr")
     .when(F.col("Mes") == 5, "Mai")
     .when(F.col("Mes") == 6, "Jun")
     .when(F.col("Mes") == 7, "Jul")
     .when(F.col("Mes") == 8, "Ago")
     .when(F.col("Mes") == 9, "Set")
     .when(F.col("Mes") == 10, "Out")
     .when(F.col("Mes") == 11, "Nov")
     .otherwise("Dez")
)


# NOMES DOS DIAS DA SEMANA (PT-BR)

df_cal = df_cal.withColumn(
    "NomeDiaSemana",
    F.when(F.col("DiaSemana") == 1, "Domingo")
     .when(F.col("DiaSemana") == 2, "Segunda-feira")
     .when(F.col("DiaSemana") == 3, "Terça-feira")
     .when(F.col("DiaSemana") == 4, "Quarta-feira")
     .when(F.col("DiaSemana") == 5, "Quinta-feira")
     .when(F.col("DiaSemana") == 6, "Sexta-feira")
     .otherwise("Sábado")
)

df_cal = df_cal.withColumn(
    "NomeDiaSemanaAbrev",
    F.when(F.col("DiaSemana") == 1, "Dom")
     .when(F.col("DiaSemana") == 2, "Seg")
     .when(F.col("DiaSemana") == 3, "Ter")
     .when(F.col("DiaSemana") == 4, "Qua")
     .when(F.col("DiaSemana") == 5, "Qui")
     .when(F.col("DiaSemana") == 6, "Sex")
     .otherwise("Sáb")
)


# Salvar Tabela Calendário

df_cal.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("default.dim_calendario")

# Visualização Tabela Calendário

display(df_cal.limit(10))


Data,Ano,Mes,Dia,DiaSemana,Trimestre,SemanaAno,AnoMes,AnoSemana,NomeMes,NomeMesAbrev,NomeDiaSemana,NomeDiaSemanaAbrev
2000-01-01,2000,1,1,7,1,52,2000-01,2000-52,Janeiro,Jan,Sábado,Sáb
2000-01-02,2000,1,2,1,1,52,2000-01,2000-52,Janeiro,Jan,Domingo,Dom
2000-01-03,2000,1,3,2,1,1,2000-01,2000-1,Janeiro,Jan,Segunda-feira,Seg
2000-01-04,2000,1,4,3,1,1,2000-01,2000-1,Janeiro,Jan,Terça-feira,Ter
2000-01-05,2000,1,5,4,1,1,2000-01,2000-1,Janeiro,Jan,Quarta-feira,Qua
2000-01-06,2000,1,6,5,1,1,2000-01,2000-1,Janeiro,Jan,Quinta-feira,Qui
2000-01-07,2000,1,7,6,1,1,2000-01,2000-1,Janeiro,Jan,Sexta-feira,Sex
2000-01-08,2000,1,8,7,1,1,2000-01,2000-1,Janeiro,Jan,Sábado,Sáb
2000-01-09,2000,1,9,1,1,1,2000-01,2000-1,Janeiro,Jan,Domingo,Dom
2000-01-10,2000,1,10,2,1,2,2000-01,2000-2,Janeiro,Jan,Segunda-feira,Seg
